# Strava segment data
TODO: Get segment id

In [21]:
import helium
import pandas as pd
import os
import time
def metadata(**kwargs):
    return kwargs
db_dir='db'
db_file='{}/strava.json'.format(db_dir)

In [ ]:
os.makedirs(db_dir,exist_ok=True)

if os.path.isfile(db_file):
    df=pd.read_json(db_file)
else:
    df=pd.DataFrame()

browser=helium.start_chrome("https://www.strava.com/login")

helium.write('diego.restrepo@gmail.com',into='Your Email')

helium.write('opoqdh11',into='Password')

helium.click('Log In')

time.sleep(10)
browser.maximize_window()

In [ ]:
#Required to activate card-section below
lt=[t for t in browser.find_elements_by_class_name('card-section') 
   if t.text.find('Latest Activity')>-1][0]

lt.find_element_by_tag_name('a').click()

# Web page with latest activity

# General info

In [19]:
time.sleep(10)
dt=browser.find_element_by_class_name('details')

date=dt.find_element_by_tag_name('time').text

name=dt.find_element_by_tag_name('h1').text

kk=browser.find_elements_by_tag_name("ul")

q=[ k for k in kk if k.text.find('\nDistance')>-1][0]

distance,time,elevation=(t.text for t in q.find_elements_by_tag_name('strong') )

q=[ k for k in kk if k.text.find('\nEstimated Avg Power')>-1][0]

power,energy=(t.text for t in q.find_elements_by_tag_name('strong') )

# Load tables

browser.find_element_by_id('show-hidden-efforts').click()

html=browser.page_source

ll=pd.read_html(html)

# General info table

mtdt=ll[0].groupby('Unnamed: 0').sum().transpose().to_dict()

mtdt.update(  metadata(date=date,name=name,distance=distance,time=time,
         elevation=elevation,power=elevation,energy=energy)
   )

#Fix time
import time

# Segments

sgmt=ll[1][['Name','Time','Speed','Power']]

sgmt=sgmt.append( ll[2].rename({2:'Name',4:'Time',5:'Speed',6:'Power'},
            axis='columns')[['Name','Time','Speed','Power']] 
           ).reset_index(drop=True)

sgmt['Info']=''

sgmt['Info'] = [ mtdt  for i in sgmt.index]



KeyError: "None of [Index(['Name', 'Time', 'Speed', 'Power'], dtype='object')] are in the [columns]"

In [20]:
sgmt

Name                     \
                                                  MAX Unnamed: 3_level_1   
                                                 Time Unnamed: 3_level_2   
0   La Vaquita - Torre Eiffel (Sabaneta)  0.46km  ...                NaN   
1   La Vaquita - La Octava Maravilla  1.64km  240m...                NaN   
2   La Vaquita - La Tuna - descenso Del Taburete -...                NaN   
3                                                 MAX                NaN   
4                                            76.0km/h                NaN   
5                                                   0                NaN   
6                                              0  bpm                NaN   
7                                                   —                NaN   
8                                                 NaN                NaN   
9                                                Time                NaN   
10           La Vaquita - Monteflor  0.80km  91m  11%                NaN   
11                 carmelo-la tuna  2.44km  365m  15%                NaN   
12  San Judas hasta destapada (Via MonteCarmelo)  ...                NaN   
13                       San Judas  1.43km  240m  17%                NaN   
14  San Judas 0.7 Km Finales al 23%  0.71km  165m ...                NaN   
15  Via MonteCarmelo (destapada) hasta La Tuna (Mi...                NaN   
16  Via MonteCarmelo - Remate Tuna  0.89km  116m  13%                NaN   
17                   Remate La Tuna  0.32km  41m  13%                NaN   
18      Lomita Aves María - Espigas  0.52km  52m  10%                NaN   
19             La doctora - Lomitas  0.33km  41m  12%                NaN   
20             La doctora - Lomitas  0.33km  41m  12%                NaN   
21                 lomita avesmaría  0.16km  15m  10%                NaN   

                 Time              Speed              Power  
   Unnamed: 4_level_1 Unnamed: 5_level_1 Unnamed: 6_level_1  
   Unnamed: 4_level_2 Unnamed: 5_level_2 Unnamed: 6_level_2  
0                4:30            6.2km/h               212W  
1               25:20            3.9km/h               183W  
2             1:01:11            6.7km/h               162W  
3                 NaN                NaN                NaN  
4                 NaN                NaN                NaN  
5                 NaN                NaN                NaN  
6                 NaN                NaN                NaN  
7                 NaN                NaN                NaN  
8                 NaN                NaN                NaN  
9                 NaN                NaN                NaN  
10               9:26            5.1km/h               186W  
11              42:14            3.5km/h               174W  
12              26:46            2.7km/h               179W  
13              31:29            2.7km/h               161W  
14              16:10            2.6km/h               181W  
15               8:13            4.2km/h               137W  
16              13:52            3.9km/h               167W  
17               4:28            4.4km/h               214W  
18               6:49            4.6km/h               185W  
19               4:59            4.0km/h               187W  
20               4:59            4.0km/h               187W  
21               2:50            3.4km/h               162W

In [14]:
df=df.append(sgmt,sort=False).reset_index(drop=True)

In [15]:
df.to_json(db_file)

In [16]:
f=open('commit.sh','w')
f.write('git commit -am "{}"\n'.format(
    df.iloc[-1]['Info']['name']+' '+df.iloc[-1]['Info']['date']
))
f.write('git push origin master')
f.close()

In [17]:
!bash commit.sh

[master 9185133] La Tuna 2:24 PM on Sunday, May 3, 2020
 3 files changed, 252 insertions(+), 48 deletions(-)
Counting objects: 6, done.
Delta compression using up to 8 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (6/6), 148.68 KiB | 0 bytes/s, done.
Total 6 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To github.com:restrepo/strava.git
   e52c016..9185133  master -> master
